In [ ]:
%%html
<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at 'In [9]'.</span>

An Exception was encountered at 'In [9]'.

In [1]:
# parameters section

pdb_file = ''
ligands_file = ''
ref_file = ''
conf_file = ''
docking_directory = ''

In [2]:
# Parameters
pdb_file = "/data/GDP_frags/5O33_nolig.pdb"
ligands_file = "/data/GDP_frags/GDP_frag1/250_surprise_sucos.sdf"
ref_file = "/data/GDP_frags/GDP_aligned.sdf"
conf_file = "conf.txt"
docking_directory = "/data/docking/gdp_frag-1/surpise_set"


In [3]:
import sys
sys.path.append('/code')
from tools.misc import is_interactive

is_interactive()

False

In [4]:
import warnings
warnings.filterwarnings('ignore')
import glob
from tools.docking.vina import vina_dock_multi
from tools.docking.preparation import define_vina_box
from example_notebooks.ui.file_widgets import PathSelector, DockButtons, InputBoxSet
import os, shutil

# Docking (vina) multiple ligands in a box defined by a reference ligand
---

### 1. Selecting some inputs:

---

**First we need to select some inputs, namely:**
- the protein we want to dock into (pdb file)
- the reference ligand to define a box (mol/sdf)
- the ligands we want to dock (sdf)
- tan output directory

**To do this:**  
*(N.B. run the next 2 cells to show the file dialogues)*
1. click on the arrow in the dropdown box at the top
2. select the relevant file in the dialogue 
3. click the button next to either protein, query ligands or reference to set the file

*repeat this until all files are selected*

4. type the directory you want to get the results of the docking in into the 'Docking dir' input
5. select 'Set' to set the directory if the directory exists or create to create it, and then set to set it



In [5]:
if is_interactive():
    f = PathSelector(start_dir=os.path.join(os.getcwd()))
    display(f.accord)
    b = DockButtons(f)
    display(b.render_buttons())

In [6]:
if is_interactive():
    i = InputBoxSet(tlabel='Docking dir:')
    display(i.render())

---

### 2. Check the inputs 
---

Run the next cell, and check you are happy with the inputs and docking directory


In [7]:
if is_interactive():
    pdb_file = b.pdb_button.description
    ligands_file = b.lig_button.description
    ref_file = b.ref_button.description
    conf_file = 'conf.txt'
    docking_directory = i.inbox.value
else:
    if not os.path.isdir(docking_directory):
        os.makedirs(docking_directory)    

print(f'inputs: \n\t - pdb: {pdb_file} \n\t - ligs: {ligands_file} \
      \n\t - ref: {ref_file} \n\ndocking directory: {docking_directory}')

inputs: 
	 - pdb: /data/GDP_frags/5O33_nolig.pdb 
	 - ligs: /data/GDP_frags/GDP_frag1/250_surprise_sucos.sdf       
	 - ref: /data/GDP_frags/GDP_aligned.sdf 

docking directory: /data/docking/gdp_frag-1/surpise_set


---

### 3. Define a box from the reference ligand
---

Define a box around your reference ligand, and then write it out to a configuration file for autodock vina

There are some parameters that you can change here:

```
define_vina_box:
    Use a reference ligand to produce a box definition for autodock vina
    
        :param ref_lig: a mol or sdf file of the reference ligand
        :param output: the name of the output file
        :param bufx: distance (A) of buffer in x direction (default=10)
        :param bufy: distance (A) of buffer in y direction (default=10)
        :param bufz: distance (A) of buffer in z direction (default=10)
        :param exhaustiveness: exhaustivness of the vina calculation (default=20)
        :param seed: random seed for vina (default=None)
        :param cpu: number of cpu's to use in vina calculation
```

In [8]:
define_vina_box(ligands_file, conf_file)

---

### 4. Dock!
---

There are some parameters that you can change here:

```
vina_dock_multi:
    Dock multiple ligands from an sdf file against a protein, with a given configuration file with vina. 
    Also does preparation from pdb and sdf
    
        :param protein_pdb: the input pdb file to prepare and dock against
        :param ligands_sdf: an sdf file of multiple ligands to split, prepare and dock
        :param conf: the path of a configuration file (required for now)
        :param docking_dir: the output directory
        :param multiproc: use multi-processing? (boolean)
        :param exe: the path to the vina executable
```

But if you have followed the steps in the above boxes, you can just run this cell without a worry!


In [9]:
vina_dock_multi(protein_pdb=pdb_file, ligands_sdf=ligands_file, 
                conf=conf_file, docking_dir=docking_directory, multiproc=True)

PermissionError: [Errno 13] Permission denied: '/data/docking/gdp_frag-1/surpise_set/250_surprise_sucos.sdf'

In [ ]:
docking_dir=docking_directory

if not os.path.isdir(os.path.join(docking_dir, 'docked/')):
    os.mkdir(os.path.join(docking_dir, 'docked/'))
if not os.path.isdir(os.path.join(docking_dir, 'logs/')):
    os.mkdir(os.path.join(docking_dir, 'logs/'))
if not os.path.isdir(os.path.join(docking_dir, 'inputs/')):
    os.mkdir(os.path.join(docking_dir, 'inputs/'))

docked_files = glob.glob(os.path.join(docking_dir, '*_docked.pdbqt'))
for f in docked_files:
    shutil.move(f, os.path.join(docking_dir, 'docked/' + f.split('/')[-1]))

logs = glob.glob(os.path.join(docking_dir, '*_docked.log'))
for f in logs:
    shutil.move(f, os.path.join(docking_dir, 'logs/' + f.split('/')[-1]))

in_ligs = glob.glob(os.path.join(docking_dir, '*_prepared.pdbqt'))
for f in in_ligs:
    shutil.move(f, os.path.join(docking_dir, 'inputs/' + f.split('/')[-1]))

print_str = f"Docking complete! Your results have been saved to: \
\n\t - docked ligands: {os.path.join(docking_directory, 'docked/')} \
\n\t - log files: {os.path.join(docking_directory, 'logs/')} \
\n\t - prepared input files: {os.path.join(docking_directory, 'inputs/')}"
        
print(print_str)